In [1]:
%run 'SQLEngine.ipynb'

In [2]:
%run 'SpotifyEngine.ipynb'

In [3]:
target_artist = 'Dua Lipa'

# 1. Get the Current Data in the DB

In [4]:
related_artist_qry = """

select related_a_name.artistname "Name"
from artists target_a

inner join related_artists related_a
on related_a.targetaid = target_a.aid

inner join artists related_a_name
on related_a_name.aid = related_a.relatedaid

where target_a.artistname = 'Dua Lipa'

"""

In [5]:
#login info
host = "concertmanager.postgres.database.azure.com"
dbname = "ConcertManager"
user = "romitbarua@concertmanager"
password = "concertManager1234!"
sslmode = "require"
engine = SQLEngine(host, dbname, user, password, sslmode)

In [6]:
engine.query(related_artist_qry)

[('Years & Years',)]

# 2. Check if Any of the Current Related Artists have Concerts Coming Up

# 3. Pull New Related Artists from Spotify

In [7]:
client_id = '868cd5aefb524008bee01a4c9b3feba1'
client_secret = '50d9e80711934c858057e98af05676d9'
redirect_uri = 'http://localhost:8080/callback'
username = '1npp6j4y6ynf7jrkvwepadbyk'
scope = 'playlist-modify-private'

In [8]:
spotifyEngine = Spotify(client_id, client_secret, redirect_uri, username, scope)

In [9]:
related_df = spotifyEngine.getRelatedArtists(target_artist)

In [10]:
related_df

,TargetArtistName,TargetArtistSpotifyID,RelatedArtistName,RelatedArtistSpotifyID
0,Dua Lipa,6M2wZ9GZgrQXHCFfjv46we,Years & Years,5vBSrE1xujD2FXYRarbAXc
0,Dua Lipa,6M2wZ9GZgrQXHCFfjv46we,Rita Ora,5CCwRZC6euC8Odo6y9X8jr
0,Dua Lipa,6M2wZ9GZgrQXHCFfjv46we,Selena Gomez,0C8ZW7ezQVs4URX5aX7Kqx
0,Dua Lipa,6M2wZ9GZgrQXHCFfjv46we,Ava Max,4npEfmQ6YuiwW1GpUmaq3F
0,Dua Lipa,6M2wZ9GZgrQXHCFfjv46we,Fifth Harmony,1l8Fu6IkuTP0U5QetQJ5Xt
0,Dua Lipa,6M2wZ9GZgrQXHCFfjv46we,Camila Cabello,4nDoRrQiYLoBzwC5BhVJzF
0,Dua Lipa,6M2wZ9GZgrQXHCFfjv46we,Zara Larsson,1Xylc3o4UrD53lo9CvFvVg
0,Dua Lipa,6M2wZ9GZgrQXHCFfjv46we,Ellie Goulding,0X2BH1fck6amBIoJhDVmmJ
0,Dua Lipa,6M2wZ9GZgrQXHCFfjv46we,Jess Glynne,4ScCswdRlyA23odg9thgIO
0,Dua Lipa,6M2wZ9GZgrQXHCFfjv46we,DNCE,6T5tfhQCknKG4UnH90qGnz


# 4. Upload Artist and Related Artist Data

In [11]:
def uploadRelatedArtistRow(row, SQLEngine):
    SQLEngine.executeSP('add_related_artists', list(row))
    

def uploadRelatedArtist(related_df, SQLEngine):
    related_df.apply(lambda rw:uploadRelatedArtistRow(rw, SQLEngine), axis=1)

In [12]:
uploadRelatedArtist(related_df, engine)

In [ ]:
def genFeatures(data):

    data['hasFace'] = data['email'].str.contains('face=').astype('int')
    data['hasSize'] = data['email'].str.contains('size=').astype('int')
    data['hasWidth'] = data['email'].str.contains('width=').astype('int')
    data['hasBlockquote'] = data['email'].str.contains('blockquote').astype('int')
    data['isReply'] = data['subject'].str.contains('re:').astype('int')
    data['isForward'] = data['subject'].str.contains('Fw:').astype('int')
    data['includesSpam'] = data['email'].str.contains('spam').astype('int')
    data['includesFree'] = np.where((data.subject.str.lower().str.contains('free')) | (data.email.str.lower().str.contains('free')), 1, 0)
    data['subject$'] = data['subject'].str.contains('$').astype('int')
    data['hasAssist'] = data['subject'].str.lower().str.contains('assistance').astype('int')
    data['hasLink'] = data['email'].str.lower().str.contains('http').astype('int')
    data['hasWWW'] = data['email'].str.lower().str.contains('www.').astype('int')
    data['LinkAndWww'] = data['hasLink']*data['hasWWW']
    data['emailPlease'] = data['email'].str.lower().str.contains('plea').astype('int')
    data['subjectPlease'] = data['subject'].str.lower().str.contains('plea').astype('int')
    data['emailHTML'] = data['email'].str.lower().str.contains('html').astype('int')

    data['numSubjectChar'] = data['subject'].str.len()
    data['numEmailChar'] = data['email'].str.len()

    count = lambda l1,l2: sum([1 for x in l1 if x in l2])
    data['emailPunc'] = data.email.apply(lambda s: count(s, string.punctuation))
    data['subjectPunc'] = data.subject.apply(lambda s: count(s, string.punctuation))
    data['subject!'] = data.subject.apply(lambda s: count(s, '!'))
    data['email!'] = data.email.apply(lambda s: count(s, '!'))

    cap_regex = '[A-Z][A-Z]+'
    data['consecutiveCaps'] = data.subject.str.contains(cap_regex).astype('int')
    pattern2 = '[1-9][1-9]+'
    data['subjectNum'] = data.subject.str.contains(pattern2).astype('int')

    data['emailPuncPerc'] = data['emailPunc']/data['numEmailChar']

    longest_word = []
    for i in range(len(data)):
        longest_word.append(len(max(data.loc[i, 'email'].split(), key=len)))

    data['longest_word'] = longest_word